In [ ]:
!pip install gradio groq diffusers transformers accelerate safetensors torch --quiet

In [ ]:
import gradio as gr
from groq import Groq
import torch
from diffusers import StableDiffusionPipeline
from google.colab import userdata
import os

# Enhanced API key handling
try:
    api_key = userdata.get('GROQ_API_KEY')
    if not api_key:
        raise ValueError("GROQ_API_KEY not found in secrets")
    client = Groq(api_key=api_key)
    print("✅ Groq client initialized successfully")
except Exception as e:
    print(f"❌ Error initializing Groq client: {e}")

def refine_prompt(user_prompt):
    """Send user prompt to Groq LLM to make it creative & image-ready."""
    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "You are a creative prompt engineer for product design. Transform user ideas into vivid, detailed artistic descriptions perfect for image generation. Include materials, colors, style, lighting, and composition details."},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.8,
            max_tokens=300,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error refining prompt: {str(e)}"

# Initialize Stable Diffusion
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=dtype,
).to(device)

def generate_image(prompt):
    try:
        image = pipe(prompt, guidance_scale=8.5, num_inference_steps=30).images[0]
        return image
    except Exception as e:
        return f"Image generation failed: {str(e)}"

def groq_to_sd(user_prompt):
    if not user_prompt.strip():
        return "Please enter a creative idea for your custom product!", None

    # Add loading message
    refined = refine_prompt(user_prompt)

    # Enhanced prompt for better product images
    enhanced_prompt = f"{refined}, professional product photography, clean white background, studio lighting, high quality, detailed, commercial grade, 4K"

    img = generate_image(enhanced_prompt)
    return refined, img

# Custom CSS for premium visual design
css = """
:root {
    --primary: #6366f1;
    --primary-dark: #4f46e5;
    --secondary: #ec4899;
    --secondary-dark: #db2777;
    --accent: #f59e0b;
    --dark: #1f2937;
    --light: #f9fafb;
    --gray: #9ca3af;
    --success: #10b981;
}

body {
    font-family: 'Inter', 'Segoe UI', system-ui, sans-serif;
    background: linear-gradient(135deg, #f3f4f6 0%, #e5e7eb 100%);
    margin: 0;
    padding: 0;
}

.gradio-container {
    max-width: 1200px !important;
    margin: 0 auto;
    padding: 20px;
    background: transparent !important;
}

.main-container {
    display: flex;
    flex-direction: column;
    gap: 24px;
}

.header {
    text-align: center;
    padding: 30px 20px;
    background: linear-gradient(135deg, var(--primary) 0%, var(--primary-dark) 100%);
    border-radius: 20px;
    color: white;
    box-shadow: 0 10px 25px rgba(99, 102, 241, 0.3);
    margin-bottom: 20px;
}

.header h1 {
    font-size: 2.8rem;
    font-weight: 800;
    margin: 0;
    letter-spacing: -0.5px;
    background: linear-gradient(to right, #ffffff, #e0e7ff);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    text-shadow: 0 2px 4px rgba(0,0,0,0.1);
}

.header p {
    font-size: 1.2rem;
    opacity: 0.9;
    max-width: 700px;
    margin: 10px auto 0;
    font-weight: 400;
}

.content {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 24px;
}

.input-panel, .output-panel {
    background: white;
    border-radius: 20px;
    padding: 30px;
    box-shadow: 0 4px 20px rgba(0, 0, 0, 0.08);
    border: 1px solid rgba(255, 255, 255, 0.6);
}

.panel-title {
    font-size: 1.5rem;
    font-weight: 700;
    color: var(--dark);
    margin-bottom: 20px;
    display: flex;
    align-items: center;
    gap: 10px;
}

.panel-title::before {
    content: "";
    width: 8px;
    height: 8px;
    border-radius: 50%;
    background: var(--primary);
}

.input-container {
    display: flex;
    flex-direction: column;
    gap: 20px;
}

.textbox-container {
    position: relative;
}

.gr-textbox textarea {
    border-radius: 16px;
    border: 2px solid #e5e7eb;
    padding: 16px;
    font-size: 16px;
    min-height: 120px !important;
    transition: all 0.3s ease;
    background: #fafafa;
}

.gr-textbox textarea:focus {
    border-color: var(--primary);
    box-shadow: 0 0 0 4px rgba(99, 102, 241, 0.1);
    background: white;
    outline: none;
}

.examples-container {
    margin-top: 10px;
}

.examples-title {
    font-weight: 600;
    color: var(--dark);
    margin-bottom: 12px;
    font-size: 1.1rem;
}

.examples-grid {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 12px;
}

.example-chip {
    background: #f3f4f6;
    padding: 12px 16px;
    border-radius: 12px;
    font-size: 0.9rem;
    cursor: pointer;
    transition: all 0.2s ease;
    border: 1px solid #e5e7eb;
}

.example-chip:hover {
    background: var(--primary);
    color: white;
    transform: translateY(-2px);
    box-shadow: 0 4px 12px rgba(99, 102, 241, 0.3);
}

.generate-btn {
    background: linear-gradient(45deg, var(--primary), var(--primary-dark));
    color: white;
    border: none;
    border-radius: 14px;
    padding: 16px 32px;
    font-weight: 600;
    font-size: 1.1rem;
    transition: all 0.3s ease;
    box-shadow: 0 4px 15px rgba(99, 102, 241, 0.4);
    width: 100%;
    margin-top: 10px;
    cursor: pointer;
}

.generate-btn:hover {
    transform: translateY(-2px);
    box-shadow: 0 6px 20px rgba(99, 102, 241, 0.5);
}

.output-content {
    display: flex;
    flex-direction: column;
    gap: 20px;
}

.prompt-output {
    background: #f8fafc;
    border-radius: 16px;
    padding: 20px;
    border: 1px solid #e2e8f0;
    font-size: 0.95rem;
    line-height: 1.6;
    max-height: 200px;
    overflow-y: auto;
}

.image-output {
    border-radius: 16px;
    overflow: hidden;
    box-shadow: 0 4px 20px rgba(0, 0, 0, 0.1);
    background: #f1f5f9;
    min-height: 400px;
    display: flex;
    align-items: center;
    justify-content: center;
    color: var(--gray);
}

.footer {
    text-align: center;
    padding: 20px;
    color: var(--gray);
    font-size: 0.9rem;
    margin-top: 20px;
}

.loading {
    display: inline-block;
    width: 20px;
    height: 20px;
    border: 3px solid rgba(99, 102, 241, 0.3);
    border-radius: 50%;
    border-top-color: var(--primary);
    animation: spin 1s ease-in-out infinite;
}

@keyframes spin {
    to { transform: rotate(360deg); }
}

@media (max-width: 768px) {
    .content {
        grid-template-columns: 1fr;
    }

    .header h1 {
        font-size: 2.2rem;
    }

    .examples-grid {
        grid-template-columns: 1fr;
    }
}
"""

# Create the interface with a custom layout
with gr.Blocks(css=css, title="Weavelancer - AI Product Designer") as demo:
    gr.HTML("""
    <div class="header">
        <h1>🧵 Weavelancer - AI Product Designer</h1>
        <p>Transform your ideas into stunning product designs with the power of AI</p>
    </div>
    """)

    with gr.Row(elem_classes="main-container"):
        with gr.Column(scale=1, elem_classes="input-panel"):
            gr.HTML("""
            <div class="panel-title">
                <span>💡 Your Product Vision</span>
            </div>
            """)

            prompt_input = gr.Textbox(
                label="",
                placeholder="Describe your product idea in detail...\nExample: Minimalist eco-friendly scarf with geometric patterns, sustainable materials, pastel colors",
                lines=5,
                max_lines=5,
                elem_classes="textbox-container"
            )

            gr.HTML("""
            <div class="examples-container">
                <div class="examples-title">💎 Try these examples:</div>
                <div class="examples-grid">
                    <div class="example-chip">Handwoven bohemian tapestry with mandala patterns</div>
                    <div class="example-chip">Modern ceramic coffee mug with geometric gold accents</div>
                    <div class="example-chip">Sustainable bamboo phone case with nature design</div>
                    <div class="example-chip">Vintage-inspired leather journal with copper hardware</div>
                </div>
            </div>
            """)

            generate_btn = gr.Button("Generate Design →", elem_classes="generate-btn")

        with gr.Column(scale=1, elem_classes="output-panel"):
            gr.HTML("""
            <div class="panel-title">
                <span>✨ AI-Enhanced Design</span>
            </div>
            """)

            with gr.Column(elem_classes="output-content"):
                prompt_output = gr.Textbox(
                    label="Creative Brief",
                    lines=6,
                    max_lines=10,
                    show_copy_button=True,
                    elem_classes="prompt-output"
                )

                image_output = gr.Image(
                    label="Product Mockup",
                    height=400,
                    width=400,
                    elem_classes="image-output"
                )

    gr.HTML("""
    <div class="footer">
        <p>Powered by Groq AI and Stable Diffusion | Crafted with ❤️ for creators</p>
    </div>
    """)

    # Add JavaScript for example chips
    gr.HTML("""
    <script>
    document.querySelectorAll('.example-chip').forEach(chip => {
        chip.addEventListener('click', () => {
            const textarea = document.querySelector('textarea');
            if (textarea) {
                textarea.value = chip.textContent;
                textarea.dispatchEvent(new Event('input', { bubbles: true }));
            }
        });
    });
    </script>
    """)

    generate_btn.click(
        fn=groq_to_sd,
        inputs=[prompt_input],
        outputs=[prompt_output, image_output]
    )

# Launch the application
if __name__ == "__main__":
    demo.launch(
        debug=True,
        share=True,
        show_error=True,
        inbrowser=True
    )